In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

# url = "https://www.espncricinfo.com/records/tournament/team-match-results/icc-men-s-t20-world-cup-2024-15946"

# response=requests.get(url)
# response.raise_for_status()

In [ ]:
# soup= BeautifulSoup(response.text,"html.parser")

In [ ]:
# data=[]
# table_body= soup.find("tbody")
# if table_body :
#   print("Table is present")
# rows= table_body.find_all("tr")
# len(rows)
# for row in rows:
#   cells = row.find_all('td')
#   team1 = cells[0].text.strip()
#   team2 = cells[1].text.strip()
#   winner= cells[2].text.strip()
#   margin= cells[3].text.strip()
#   ground= cells[4].text.strip()
#   match_date= cells[5].text.strip()
#   score_card= cells[6].text.strip()
#   data.append({"team1":team1, "team2":team2, "winner": winner,"margin":margin,"ground":ground, "match_date":match_date,"score_card":score_card})

# with open('table_data.json','w',encoding='utf-8' ) as f:
#     json.dump(data,f,ensure_ascii=False,indent=4)

In [ ]:
with open('/content/t20_wc_match_results.json') as f:
  data=json.load(f)
data[0]['matchSummary']

[{'team1': 'Namibia',
  'team2': 'Sri Lanka',
  'winner': 'Namibia',
  'margin': '55 runs',
  'ground': 'Geelong',
  'matchDate': 'Oct 16, 2022',
  'scorecard': 'T20I # 1823'},
 {'team1': 'Netherlands',
  'team2': 'U.A.E.',
  'winner': 'Netherlands',
  'margin': '3 wickets',
  'ground': 'Geelong',
  'matchDate': 'Oct 16, 2022',
  'scorecard': 'T20I # 1825'},
 {'team1': 'Scotland',
  'team2': 'West Indies',
  'winner': 'Scotland',
  'margin': '42 runs',
  'ground': 'Hobart',
  'matchDate': 'Oct 17, 2022',
  'scorecard': 'T20I # 1826'},
 {'team1': 'Ireland',
  'team2': 'Zimbabwe',
  'winner': 'Zimbabwe',
  'margin': '31 runs',
  'ground': 'Hobart',
  'matchDate': 'Oct 17, 2022',
  'scorecard': 'T20I # 1828'},
 {'team1': 'Namibia',
  'team2': 'Netherlands',
  'winner': 'Netherlands',
  'margin': '5 wickets',
  'ground': 'Geelong',
  'matchDate': 'Oct 18, 2022',
  'scorecard': 'T20I # 1830'},
 {'team1': 'Sri Lanka',
  'team2': 'U.A.E.',
  'winner': 'Sri Lanka',
  'margin': '79 runs',
  'gr

In [ ]:
df_match=pd.DataFrame(data[0]["matchSummary"])
df_match.head()

,team1,team2,winner,margin,ground,matchDate,scorecard
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830


In [ ]:
df_match.rename({'scorecard':'match_id'}, inplace=True, axis=1)
df_match.head()

,team1,team2,winner,margin,ground,matchDate,match_id
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830


In [ ]:
df_match.to_csv('dim_match_summary.csv',index=False)

**Create a match ids dictionary that maps team names to a unique match id. This will be useful later on to link with other tables**

In [ ]:
match_id_dict={}
for index,row in df_match.iterrows():
  key1 =row['team1'] + ' Vs ' + row['team2']
  key2 = row['team2']+ ' Vs '+ row['team1']
  match_id_dict[key1]= row['match_id']
  match_id_dict[key2]= row['match_id']


**Batting Summary**

In [ ]:
with open('/content/t20_wc_batting_summary.json') as f:
  batt_data=json.load(f)
all_records=[]
for rec in batt_data:
  all_records.extend(rec['battingSummary'])

df_batting=pd.DataFrame(all_records)


In [ ]:
df_batting["out/not_out"]= df_batting["dismissal"].apply(lambda x: "out" if len(x)>0 else "not_out")

In [ ]:
df_batting.drop(columns=["dismissal"],inplace=True)

In [ ]:
df_batting.shape

(699, 10)

In [ ]:
df_batting['match_id'] = df_batting['match'].map(match_id_dict)
df_batting

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,out/not_out,match_id
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50.00,out,T20I # 1823
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100.00,out,T20I # 1823
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,out,T20I # 1823
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,out,T20I # 1823
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),20,24,0,0,83.33,out,T20I # 1823
...,...,...,...,...,...,...,...,...,...,...,...
694,Pakistan Vs England,England,3,Phil Salt,10,9,2,0,111.11,out,T20I # 1879
695,Pakistan Vs England,England,4,Ben Stokes,52,49,5,1,106.12,not_out,T20I # 1879
696,Pakistan Vs England,England,5,Harry Brook,20,23,1,0,86.95,out,T20I # 1879
697,Pakistan Vs England,England,6,Moeen Ali,19,13,3,0,146.15,out,T20I # 1879


In [ ]:
df_batting['batsmanName']= df_batting['batsmanName'].apply(lambda x : x.replace("†",''))
# df_batting['batsmanName']= df_batting['batsmanName'].apply(lambda x : x.replace("†",''))

In [ ]:
df_batting.to_csv('t20_wc_batting_summary.csv',index=False)

In [ ]:
df_batting.shape

(699, 11)

**Process Bowling Summary**

In [ ]:
with open("/content/t20_wc_bowling_summary.json") as f:
  bowl_data=json.load(f)
bowling_record=[]
for record in  bowl_data:
    bowling_record.extend(record['bowlingSummary'])

df_bowling= pd.DataFrame(bowling_record)


In [ ]:
df_bowling.shape

(500, 13)

In [ ]:
df_bowling.head()

,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls
0,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0
1,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0
2,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0
3,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0
4,Namibia Vs Sri Lanka,Sri Lanka,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0


In [ ]:
df_bowling['match_id']= df_bowling['match'].map(match_id_dict)

In [ ]:
df_bowling.to_csv('t20_wc_bowling_summary.csv', index=False)

**Process Players Information**

In [ ]:
with open('/content/t20_wc_player_info.json') as f:
  player_data=json.load(f)
df_player=pd.DataFrame(player_data)
df_player.head()


,name,team,battingStyle,bowlingStyle,playingRole,description
0,Michael van Lingen,Namibia,Left hand Bat,Left arm Medium,Bowling Allrounder,
1,Divan la Cock,Namibia,Right hand Bat,Legbreak,Opening Batter,
2,Jan Nicol Loftie-Eaton,Namibia,Left hand Bat,"Right arm Medium, Legbreak",Batter,
3,Stephan Baard,Namibia,Right hand Bat,Right arm Medium fast,Batter,
4,Gerhard Erasmus(c),Namibia,Right hand Bat,Right arm Offbreak,Allrounder,


In [ ]:
df_player.shape

(219, 6)

In [ ]:
df_player['name']= df_player['name'].apply(lambda x :x.replace('†',''))

In [ ]:
df_player[df_player['team']=='India']

,name,team,battingStyle,bowlingStyle,playingRole,description
127,KL Rahul,India,Right hand Bat,,Opening Batter,"A tall, elegant right-hand batsman who can kee..."
128,Rohit Sharma(c),India,Right hand Bat,Right arm Offbreak,Top order Batter,"Languid and easy on the eye, Rohit Sharma owne..."
129,Virat Kohli,India,Right hand Bat,Right arm Medium,Top order Batter,India has given to the world many a great cric...
130,Suryakumar Yadav,India,Right hand Bat,"Right arm Medium, Right arm Offbreak",Batter,Hard-hitting 360-degree batter Suryakumar Yada...
131,Axar Patel,India,Left hand Bat,Slow Left arm Orthodox,Bowling Allrounder,Left-arm spinner Axar Patel has been increasin...
132,Hardik Pandya,India,Right hand Bat,Right arm Medium fast,Allrounder,Hardik Pandya swears by living life king size ...
133,Dinesh Karthik,India,Right hand Bat,Right arm Offbreak,Wicketkeeper Batter,Not many would forget the sight of Dinesh Kart...
134,Ravichandran Ashwin,India,Right hand Bat,Right arm Offbreak,Bowling Allrounder,R Ashwin took the tricks and skills he learned...
135,Bhuvneshwar Kumar,India,Right hand Bat,Right arm Medium,Bowler,"At the time of his India debut in 2012, Bhuvne..."
136,Arshdeep Singh,India,Left hand Bat,Left arm Medium fast,Bowler,


In [ ]:
df_player.to_csv('players_no_images.csv',index=True)